## LandSAT9 Image (Dataset Availability: 2021-10-31 to Present)
### [LandSAT9 Earth Engine Documentation](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC09_C02_T1_L2)

In [1]:
# Importing the earth engine library and geemap package
import ee
import geemap

In [2]:
# Adding an interactive map.
Map = geemap.Map(basemap='ROADMAP', center=[24, 91], zoom=7, height=600, draw = True)

# Adding the TERRAIN basemaps.
Map.add_basemap("TERRAIN")
Map

Map(center=[24, 91], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [7]:
# Taking a draw feature from the map.
feature = Map.draw_last_feature

# If drawing does not exist then taking a polygon as the default version.
if feature is None:
    geom = ee.Geometry.Polygon([[[91.78481923595828, 21.95584672623341],
                                 [91.78481923595828, 21.700881280747268],
                                 [91.93176137463016, 21.700881280747268],
                                 [91.93176137463016, 21.95584672623341]]])
    feature = ee.Feature(geom, {})

# Define the AOI.
AOI = feature.geometry()

In [8]:
# Use Landsat 8 surface reflectance data.
Landsat_image_collection = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
            # filtering image collection by AOI
            .filterBounds(AOI)
            # filtering image collection by Dates
            .filterDate('2023-01-01', '2023-12-31')
            # filtering image collection by Metadata (Metadata: Cloud cover)
           .filter(ee.Filter.lt('CLOUD_COVER',50))
           .sort('CLOUD_COVER',False))

print ('Number of landsat images in AOI: ', Landsat_image_collection.size().getInfo())

Number of landsat images in AOI:  15


In [9]:
# Apply scaling factors.
def apply_scale_factors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

In [10]:
# Function for cloud masking
def fmask(image):
    # QA_PIXEL = Pixel quality attributes generated from the CFMASK algorithm.
    # Bit 0 - Fill
    # Bit 1 - Dilated Cloud
    # Bit 2 - Cirrus
    # Bit 3 - Cloud
    # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)

    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True).updateMask(qaMask)

In [11]:
# Applying scale factor 
with_cloud_image = Landsat_image_collection.map(apply_scale_factors)

# Cloud Masked Image
cloud_masked_image = Landsat_image_collection.map(fmask)

# Specify visualization parameters.
vis_natural = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

vis_nir = {
    'bands': ['SR_B5', 'SR_B4', 'SR_B3'],
    'min': 0.0,
    'max': 0.3,
}

# Generate Earth Engine layers.
left_layer = geemap.ee_tile_layer(with_cloud_image.first(), vis_natural,  'With_Cloud')
right_layer = geemap.ee_tile_layer(cloud_masked_image.first(), vis_nir, 'Without_Cloud')

# Create a split-panel map.
Map_L9 = geemap.Map(height=500)
Map_L9.split_map(left_layer, right_layer)
Map_L9.centerObject(cloud_masked_image, 9)
Map_L9

Map(center=[21.669906279022868, 91.58967219643264], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [12]:
# Getting all image collection for AOI and calculate median and clip the final image.
median_image = cloud_masked_image.median().clip(AOI)


# Specify visualization parameters.
vis_natural = {
    'bands': ['SR_B7', 'SR_B6', 'SR_B4'],
    'min': 0.0,
    'max': 0.3,
}

# Adding image to the map.
Map_all = geemap.Map(height=500)
Map_all.addLayer(median_image, vis_natural, 'Landsat-9 composite')
Map_all.centerObject(median_image, 11)
Map_all

Map(center=[21.82834239157615, 91.85829030529537], controls=(WidgetControl(options=['position', 'transparent_b…

In [13]:
task = ee.batch.Export.image.toDrive(
            image = median_image.select(["SR_B7","SR_B6","SR_B4"]),
            description = 'LandSAT9_Median_2023',
            folder = 'LandSAT_Image',
            scale = 30)
task.start()